## Препарация

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.power import TTestIndPower

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import display, HTML

plt.style.use('ggplot')

In [ ]:
order_df = pd.read_csv("there_was_a_url")

In [ ]:
order_df.head()

,completed_at,completed_at_ts,user_id,order_id,gmv_net_of_promo,promo,order_state,new_or_repeated
0,2024-12-30 09:19:26.000,1735550366,U_aa0127e75aa4bde728445c97e3c5b15c,O_21ef63f9ac436bb61ea7b0d23e63389e,273.00,0.0,complete,repeated
1,2024-12-16 18:42:27.000,1734374547,U_a1ad028ce80d1a9bc3a5ffb40cce5b2c,O_da710ca5c9fff52f749eae1e062d1076,1282.28,0.0,complete,new
2,2024-12-10 15:28:05.000,1733844485,U_e094439a0bd07d4e3904af196576c5ca,O_01cc71017bd4f4204aa22b3b01daee1b,1928.00,0.0,complete,repeated
3,2024-12-14 17:40:52.000,1734198052,U_888a3d8e5aa4a19c40fa2443a3209f5d,O_116d3e5c9a34cfa87150a4299370fff5,1199.00,0.0,complete,repeated
4,2024-12-21 11:08:11.000,1734779291,U_3bed44851dff65c45fa7892aa03020ed,O_828aaf26577a12774b348b6b70803ae4,7707.00,0.0,complete,repeated


**Колонки в «грязном» датасете:**

*   completed_at – дата и время заказа
*   completed_at_ts – время заказа в секундах
*   user_id – идентификатор пользователя
*   order_id – идентификатор заказа
*   gmv_net_of_promo – gmv заказа за вычетом промо
*   promo – размер промо, примененного к заказу
*   order_state – статус заказа (canceled, complete, resumed)
*   new_or_repeated – первый заказ или повторный

## День 1: предпосылки к метрикам
**Задача:** отобрать заказы с `2024-11-22` по `2024-12-12` cо статусом order_state `complete` и `canceled` и посчитать пользовательские сигналы

In [ ]:
order_df['completed_at'] = pd.to_datetime(order_df['completed_at'])

# Фильтрация по требуемым параметрам
filtered_df = order_df[
    (order_df['completed_at'] >= '2024-11-22') &
    (order_df['completed_at'] <= '2024-12-12') &
    (order_df['order_state'].isin(['complete', 'canceled']))
].copy()

**В новом датафрейме определим следующие поля:**

1.   user_id — идентификатор пользователя
2.   num_of_orders — общее кол-во заказов за выбранный промежуток времени
3.   sum_gmv — общая стоимость всех заказов
4.   sum_promo — сумма промокодов за все заказы

In [ ]:
# Доп. поля для анализа
filtered_df['is_promo'] = (filtered_df['promo'] > 0).astype(int)
filtered_df['full_price'] = filtered_df['gmv_net_of_promo'] + filtered_df['promo']

In [ ]:
# Агрегация по пользователям
user_signals_df = filtered_df.groupby('user_id').agg({
    'order_id': 'count',
    'gmv_net_of_promo': 'sum',
    'promo': 'sum'
}).reset_index()

user_signals_df.columns = ['user_id', 'num_of_orders', 'sum_gmv', 'sum_promo']

In [ ]:
# Флаги для подсчета метрик
user_signals_df['is_user'] = 1
user_signals_df['is_order'] = (user_signals_df['num_of_orders'] > 0).astype(int)
user_signals_df['is_promo'] = (user_signals_df['sum_promo'] > 0).astype(int)

In [ ]:
user_signals_df.head()

,user_id,num_of_orders,sum_gmv,sum_promo,is_user,is_order,is_promo
0,U_00024bcba5638d9c0a504ef7dd58b41c,1,282.33,0.00,1,1,0
1,U_0002a234ca06e3e2d7af7d7a7789bfd3,1,317.81,123.95,1,1,1
2,U_0002c29eacf17d0799bd7ce65f9a323a,1,302.16,0.00,1,1,0
3,U_0004da139a0ba2d5487271d64d7073b5,1,2898.18,0.00,1,1,0
4,U_0007de9e6d6c913691eea121569204fb,1,3000.29,150.24,1,1,1


## День 2: дизайн A/B-теста
**Задача:** взять вчерашний пользовательский датасет (на самом деле это наш предэкспериментальный период), а именно заказы с `2024-11-22` по `2024-12-12` cо статусом order state `complete` и `canceled`. Расписать гипотезу, ключевые и приемочные метрики, а также MDE

**Гипотеза:**

Новый алгоритм рассылки промокодов эффективнее текущего: процесс реактивации стал приносить больше выручки

**Ключевые метрики:**

*   gmv_per_user — средний GMV на одного пользователя: отношение суммы GMV заказов за вычетом промо к количеству пользователей в подвыборке. Комплексная метрика, отражающая общий финансовый результат процесса реактивации
*   user_to_order — коэффициент конверсии из пользователей, получивших промокод, в пользователей, совершивших заказ. Отражает степень мотивации пользователей к возвращению в продукт

**Приемочные метрики:**

*   is_promo_per_user — отношение количества пользователей, применивших промокод, к количеству пользователей в подвыборке. Отражает то, насколько релевантной оказалась рассылка промокодов пользователям
*   is_promo_per_order_user — отношение количества пользователей, применивших промокод, к пользователям, совершившим заказ. Отражает степень того, как промо повлияло на совершение заказов. Если процент маленький, возможно, реактивация произошла не из-за рассылки
*   mean_promo — средний размер промо на одного реактивированного пользователя: сумма всех промо в заказах к количеству пользователей, совершивших заказ. Отражает то, какой средний размер промо побуждал пользователей сделать покупку
*   aov — средний чек: отношение суммы GMV заказа за вычетом промо к количеству заказов за рассматриваемый период. Показатель того, насколько крупные заказы совершали реактивированные пользователи
*   promo_to_fp — средняя скидка в заказах с примененным промокодом: отношение размера промо к полной сумме (full price) заказа. Показатель того, какую часть промо составлял от размера корзины в общем случае

### Подсчет MDE

In [ ]:
def get_signals(df, num_col, denom_col):
    numerator = df[num_col].values
    denominator = df[denom_col].values

    numerator = numerator[denominator > 0]
    return numerator

def calculate_mde(signals, alpha=0.05, power=0.8):
    sample_mean = np.mean(signals)
    sample_var = np.var(signals, ddof=1)  # выборочная дисперсия
    nobs1 = len(signals) // 2  # размер каждой группы при разделении пополам

    if sample_var <= 0:
        return None, None

    mde_standardized = TTestIndPower().solve_power(
        effect_size=None, nobs1=nobs1, alpha=alpha, power=power
    )

    mde_abs = mde_standardized * np.sqrt(sample_var)
    mde_rel = (mde_abs / sample_mean * 100) if sample_mean != 0 else None

    return mde_abs, mde_rel

In [ ]:
metrics_config = {
    'gmv_per_user': {
        'data': user_signals_df,
        'numerator': 'sum_gmv',
        'denominator': 'is_user'
    },

    'user_to_order': {
        'data': user_signals_df,
        'numerator': 'is_order',
        'denominator': 'is_user'
    },

    'is_promo_per_user': {
        'data': user_signals_df,
        'numerator': 'is_promo',
        'denominator': 'is_user'
    },

    'is_promo_per_order_user': {
        'data': user_signals_df,
        'numerator': 'is_promo',
        'denominator': 'is_order'
    },

    'mean_promo': {
        'data': user_signals_df,
        'numerator': 'sum_promo',
        'denominator': 'is_order'
    },

    'aov': {
        'data': filtered_df,
        'numerator': 'gmv_net_of_promo',
        'denominator': None  # специальная обработка для AOV
    }
}

In [ ]:
def calculate_metrics_mde(metrics_config):
    results = []
    for metric_name, config in metrics_config.items():
        data = config['data']

        if metric_name == 'aov':
            signals = data['gmv_net_of_promo'].values
            metric_value = np.mean(signals)
        else:
            signals = get_signals(data, config['numerator'], config['denominator'])
            metric_value = np.mean(signals)

        mde_abs, mde_rel = calculate_mde(signals)

        mde_abs_str = f"{mde_abs:.2f}" if mde_abs is not None else "Невозможно рассчитать"
        mde_rel_str = f"{mde_rel:.2f}%" if mde_rel is not None else "Невозможно рассчитать"

        results.append({
            'metric': metric_name,
            'value': f"{metric_value:.2f}",
            'MDE_abs': mde_abs_str,
            'MDE_rel': mde_rel_str
        })

    return pd.DataFrame(results)

mde_summary_df = calculate_metrics_mde(metrics_config)

In [ ]:
def calculate_promo_to_fp_mde(df):
    promo_orders = df[df['is_promo'] == 1].copy()
    promo_orders['promo_to_fp_ratio'] = promo_orders['promo'] / promo_orders['full_price']

    signals_promo_to_fp = promo_orders['promo_to_fp_ratio'].values
    metric_value = np.mean(signals_promo_to_fp)

    mde_abs, mde_rel = calculate_mde(signals_promo_to_fp)

    mde_abs_str = f"{mde_abs:.2f}" if mde_abs is not None else "Невозможно рассчитать"
    mde_rel_str = f"{mde_rel:.2f}%" if mde_rel is not None else "Невозможно рассчитать"

    return pd.DataFrame([{
        'metric': 'promo_to_fp',
        'value': f"{metric_value:.2f}",
        'MDE_abs': mde_abs_str,
        'MDE_rel': mde_rel_str
    }])

mde_promo_to_fp_df = calculate_promo_to_fp_mde(filtered_df)

In [ ]:
all_mde_df = pd.concat([mde_summary_df, mde_promo_to_fp_df], ignore_index=True)
display(all_mde_df)

,metric,value,MDE_abs,MDE_rel
0,gmv_per_user,2949.94,138.52,4.70%
1,user_to_order,1.00,Невозможно рассчитать,Невозможно рассчитать
2,is_promo_per_user,0.44,0.01,2.73%
3,is_promo_per_order_user,0.44,0.01,2.73%
4,mean_promo,446.49,29.61,6.63%
5,aov,2121.40,73.69,3.47%
6,promo_to_fp,0.29,0.01,2.08%


## День 3: проведение A/B-теста
**Задача:** отобрать заказы с `2024-12-13` по `2025-01-03` cо статусом order_state `complete` и `canceled`, собрать таблицу пользовательских сигналов
в А/В-тесте в разрезе экспериментальных групп, посчитать и покрасить метрики

In [ ]:
user_split_df = pd.read_csv("there_was_a_url_too")

In [ ]:
ab_filtered_df = order_df[
    (order_df['completed_at'] >= '2024-12-13') &
    (order_df['completed_at'] <= '2025-01-03') &
    (order_df['order_state'].isin(['complete', 'canceled']))
].copy()

In [ ]:
# Доп. поля для анализа
ab_filtered_df['is_promo'] = (ab_filtered_df['promo'] > 0).astype(int)
ab_filtered_df['full_price'] = ab_filtered_df['gmv_net_of_promo'] + ab_filtered_df['promo']

In [ ]:
# Агрегация по пользователям
ab_user_df = ab_filtered_df.groupby('user_id').agg({
    'order_id': 'count',
    'gmv_net_of_promo': 'sum',
    'promo': 'sum'
}).reset_index()

ab_user_df.columns = ['user_id', 'num_of_orders', 'sum_gmv', 'sum_promo']

In [ ]:
# Объединение с информацией из сплита по пользователям
ab_user_df = user_split_df.merge(ab_user_df, on='user_id', how='left')

# Нули вместо NaN
ab_user_df['num_of_orders'] = ab_user_df['num_of_orders'].fillna(0)
ab_user_df['sum_gmv'] = ab_user_df['sum_gmv'].fillna(0)
ab_user_df['sum_promo'] = ab_user_df['sum_promo'].fillna(0)

# Флаги для подсчета метрик
ab_user_df['is_user'] = 1
ab_user_df['is_order'] = (ab_user_df['num_of_orders'] > 0).astype(int)
ab_user_df['is_promo'] = (ab_user_df['sum_promo'] > 0).astype(int)

print(f"Пользователей в итоговом датафрейме: {len(ab_user_df)}")
print(f"Пользователей с заказами: {ab_user_df['is_order'].sum()}")
print(f"Пользователей без заказов: {len(ab_user_df) - ab_user_df['is_order'].sum()}")

Пользователей в итоговом датафрейме: 64269
Пользователей с заказами: 21354
Пользователей без заказов: 42915


In [ ]:
# Только заказы пользователей из эксперимента
ab_order_df = ab_filtered_df[
    (ab_filtered_df['user_id'].isin(user_split_df['user_id']))
].copy()

ab_order_df = ab_order_df.merge(user_split_df[['user_id', 'exp_group']], on='user_id', how='left')

print(f"Оформленных заказов пользователей эксперимента: {len(ab_order_df)}")

Оформленных заказов пользователей эксперимента: 33670


In [ ]:
def analyze_metric(test_signals, control_signals, metric_name):
    test_value = np.mean(test_signals)
    control_value = np.mean(control_signals)

    absolute_diff = test_value - control_value
    relative_diff = (absolute_diff / control_value * 100) if control_value != 0 else 0
    p_value = stats.ttest_ind(test_signals, control_signals).pvalue

    return {
        'metric': metric_name,
        'test_value': test_value,
        'control_value': control_value,
        'absolute_diff': absolute_diff,
        'relative_diff': relative_diff,
        'p_value': p_value
    }

In [ ]:
ab_metrics_config = {
    'gmv_per_user': {
        'data': ab_user_df,
        'numerator': 'sum_gmv',
        'denominator': 'is_user'
    },

    'user_to_order': {
        'data': ab_user_df,
        'numerator': 'is_order',
        'denominator': 'is_user'
    },

    'is_promo_per_user': {
        'data': ab_user_df,
        'numerator': 'is_promo',
        'denominator': 'is_user'
    },

    'is_promo_per_order_user': {
        'data': ab_user_df,
        'numerator': 'is_promo',
        'denominator': 'is_order'
    },

    'mean_promo': {
        'data': ab_user_df,
        'numerator': 'sum_promo',
        'denominator': 'is_order'
    },

    'aov': {
        'data': ab_order_df,
        'numerator': 'gmv_net_of_promo',
        'denominator': None  # специальная обработка для AOV
    }
}

In [ ]:
results = []

for metric_name, config in ab_metrics_config.items():
    data = config['data']

    test_data = data[data['exp_group'] == 'test']
    control_data = data[data['exp_group'] == 'control']

    if metric_name == 'aov':
        test_signals = test_data['gmv_net_of_promo'].values
        control_signals = control_data['gmv_net_of_promo'].values
    else:
        test_signals = get_signals(test_data, config['numerator'], config['denominator'])
        control_signals = get_signals(control_data, config['numerator'], config['denominator'])

    analysis = analyze_metric(test_signals, control_signals, metric_name)
    results.append(analysis)

In [ ]:
ab_promo_orders = ab_order_df[ab_order_df['is_promo'] == 1].copy()
ab_promo_orders['promo_to_fp_ratio'] = ab_promo_orders['promo'] / ab_promo_orders['full_price']

test_promo_orders = ab_promo_orders[ab_promo_orders['exp_group'] == 'test']
control_promo_orders = ab_promo_orders[ab_promo_orders['exp_group'] == 'control']

test_signals = test_promo_orders['promo_to_fp_ratio'].values
control_signals = control_promo_orders['promo_to_fp_ratio'].values

analysis = analyze_metric(test_signals, control_signals, 'promo_to_fp')
results.append(analysis)

In [ ]:
summary_df = pd.DataFrame(results)

def determine_row_color(p_val, rel_diff):
    if p_val < 0.05:
        if rel_diff > 0:
            return 'lightgreen'
        else:
            return 'lightcoral'
    else:
        return 'lightgray'

colors = []
for _, row in summary_df.iterrows():
    color = determine_row_color(row['p_value'], row['relative_diff'])
    colors.append(color)

# Более понятные описания
metric_names = {
    'gmv_per_user': 'Средний GMV на пользователя',
    'user_to_order': 'Конверсия в заказавших',
    'is_promo_per_user': 'Доля исп. промокод среди всех',
    'is_promo_per_order_user': 'Доля исп. промокод среди заказавших',
    'mean_promo': 'Средний размер промо на закавшего',
    'aov': 'Средний чек',
    'promo_to_fp': 'Средняя скидка в заказах с промо'
}

summary_df['Метрика'] = summary_df['metric'].map(metric_names).fillna(summary_df['metric'])

columns_mapping = {
    'Метрика': 'Метрика',
    'test_value': 'Тест',
    'control_value': 'Контроль',
    'absolute_diff': 'Разница',
    'relative_diff': 'Разница (%)',
    'p_value': 'P-value'
}

columns_to_show = list(columns_mapping.keys())
final_df = summary_df[columns_to_show].copy()
final_df.columns = list(columns_mapping.values())

def apply_colors(row):
    row_idx = row.name
    color = colors[row_idx]
    return [f'background-color: {color}'] * len(row)

styled_table = final_df.style.apply(apply_colors, axis=1)

format_dict = {
    'Тест': '{:.2f}',
    'Контроль': '{:.2f}',
    'Разница': '{:.2f}',
    'Разница (%)': '{:.2f}%',
    'P-value': lambda x: f"{x * 100:.2f}%"
}

styled_table = styled_table.format(format_dict)

## Единый вывод по всем этапам

**Гипотеза:**

Новый алгоритм рассылки промокодов эффективнее текущего: процесс реактивации стал приносить больше выручки

**Ключевые метрики:**

*   gmv_per_user — средний GMV на одного пользователя: отношение суммы GMV заказов за вычетом промо к количеству пользователей в подвыборке. Комплексная метрика, отражающая общий финансовый результат процесса реактивации
*   user_to_order — коэффициент конверсии из пользователей, получивших промокод, в пользователей, совершивших заказ. Отражает степень мотивации пользователей к возвращению в продукт

**Приемочные метрики:**

*   is_promo_per_user — отношение количества пользователей, применивших промокод, к количеству пользователей в подвыборке. Отражает то, насколько релевантной оказалась рассылка промокодов пользователям
*   is_promo_per_order_user — отношение количества пользователей, применивших промокод, к пользователям, совершившим заказ. Отражает степень того, как промо повлияло на совершение заказов. Если процент маленький, возможно, реактивация произошла не из-за рассылки
*   mean_promo — средний размер промо на одного реактивированного пользователя: сумма всех промо в заказах к количеству пользователей, совершивших заказ. Отражает то, какой средний размер промо побуждал пользователей сделать покупку
*   aov — средний чек: отношение суммы GMV заказа за вычетом промо к количеству заказов за рассматриваемый период. Показатель того, насколько крупные заказы совершали реактивированные пользователи
*   promo_to_fp — средняя скидка в заказах с примененным промокодом: отношение размера промо к полной сумме (full price) заказа. Показатель того, какую часть промо составлял от размера корзины в общем случае

In [ ]:
display(all_mde_df)

,metric,value,MDE_abs,MDE_rel
0,gmv_per_user,2949.94,138.52,4.70%
1,user_to_order,1.00,Невозможно рассчитать,Невозможно рассчитать
2,is_promo_per_user,0.44,0.01,2.73%
3,is_promo_per_order_user,0.44,0.01,2.73%
4,mean_promo,446.49,29.61,6.63%
5,aov,2121.40,73.69,3.47%
6,promo_to_fp,0.29,0.01,2.08%


In [ ]:
display(styled_table)

,Метрика,Тест,Контроль,Разница,Разница (%),P-value
0,Средний GMV на пользователя,1518.14,1313.10,205.04,15.61%,0.03%
1,Конверсия в заказавших,0.34,0.32,0.02,7.31%,0.00%
2,Доля исп. промокод среди всех,0.15,0.13,0.02,11.71%,0.00%
3,Доля исп. промокод среди заказавших,0.43,0.41,0.02,4.09%,1.22%
4,Средний размер промо на закавшего,496.49,447.52,48.97,10.94%,3.07%
5,Средний чек,2774.13,2622.62,151.51,5.78%,0.44%
6,Средняя скидка в заказах с промо,0.27,0.26,0.01,2.13%,8.34%


### Вывод

Гипотеза подтверждена — новый алгоритм рассылки промокодов действительно оказался эффективнее текущего. Процесс реактивации на экспериментальной подвыборке показал прирост средней выручки и конвертации пользователей в «заказавших» в сравнении со старым подходом: **+15.61%** и **+7.31%** соответственно

Полученные показатели обосоновываются более релевантной рассылкой: в экспериментальной подвыборке пользователи на **11.71%** чаще использовали промокод при оформлении заказа, при этом доля использовавших промокод среди всех, кто оформил заказ, выросла на **4.09%**. Рост этого показателя подтверждает значимость полученного промо в решении оформить заказ

В экспериментальной подвыборке средний размер промо на реактивированного пользователя был выше на **10.94%** (+48.97 ₽). Тем не менее, средний чек возрос на **5.78%** (+151.51₽), а средняя скидка в заказах с примененным промокодом осталась неизменной — на уровне **26-27%**. Следовательно, несмотря на увеличинение затрат промокампании, выручка показала пропорционально больший рост